In [38]:
import requests 
import re as regex
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid

agent1 = {}
agent2 = {}
plugin = {}

## Enter Invite Url:

In [39]:
agent1['url'] = "http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiYTIyNGFkZDAtOGExZS00NDE0LWI5ZjYtNjlmNDFmNGFlMjdkIiwgInJlY2lwaWVudEtleXMiOiBbIkRGc0pLSFdUaXNXbUE1bTlLZmExTlJMY0ZBREZXb2JjRTRQaUF2ZnhYVjZiIl0sICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovL2FnZW50Mi5sb2NhbGhvc3QiLCAibGFiZWwiOiAiQ2xpZW50IChhZG1pbikifQ=="

In [40]:
# Process invite url, delete white spaces
agent1['url'] = agent1['url'].replace(" ", "")
# Regex(substitution) to extract only the invite string from url
agent1['invite_string_b64'] = regex.sub(
           r".*(c\_i\=)", 
           r"", 
           agent1['url'])
# Decoding invite string using base64 decoder
agent1['invite_string'] = base64.b64decode(agent1['invite_string_b64'])
# Converting our invite json string into a dictionary 
agent1['invite'] = json.loads(agent1['invite_string'])

print("Decoded invitation:\n")
pp.pprint(agent1['invite'])

Decoded invitation:

{'@id': 'a224add0-8a1e-4414-b9f6-69f41f4ae27d',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['DFsJKHWTisWmA5m9Kfa1NRLcFADFWobcE4PiAvfxXV6b'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [41]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

# print(base58.b58encode(did))
# print(plugin['did'])
print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 2uwZH4SDYisXdBRHMQVAT4SSNjxjJSSqYzdRSP1aZRRqouYAYbB3x6uncZuT1ZHTveayvwjHvRyWrxHuKQPiPCFd

Public key: 
 H2MSzDgvProbjzd9AE8vTihgcCERHowNfxwRMbruUY2o


**Sending a connection request to acapy**

In [42]:
uniqueId = str(uuid.uuid4())
# print(uniqueId)

# our request body
message = {
        "@id":  uniqueId,
        "~transport": {
          "return_route": "all"
        },
        "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/request",
        "label": "Plugin",
        "connection": {
          "DID": plugin['did'],
          "DIDDoc": {
            "@context": "https://w3id.org/did/v1",
            "id": plugin['did'],
            "publicKey": [{
              "id": plugin['did'] + "#keys-1",
              "type": "Ed25519VerificationKey2018",
              "controller": plugin['did'],
              "publicKeyBase58": plugin['public_key_b58']
            }],
            "service": [{
              "id": plugin['did'] + ";indy",
              "type": "IndyAgent",
              "recipientKeys": plugin['public_key_b58'],
              "serviceEndpoint": ""
            }]
          }
        }
      }

**Encoding/packing the request**

In [43]:
# encoding it with aries crypto function using the key that was
# given to us by aca-py in recipientKeys
decodedAcapyKey = base58.b58decode(agent1['invite']['recipientKeys'][0])
ourPrivateKey = plugin['keypair'][1]
# print(plugin['did'])
encodedMessage = \
    crypto.encode_pack_message(json.dumps(message), [decodedAcapyKey], ourPrivateKey)

encodedMessage = encodedMessage.decode("ascii")
# print("Encoded message: \n")
# pp.pprint(encodedMessage)

In [44]:
connectionRequestResponse = requests.post(agent1['invite']['serviceEndpoint'], data=encodedMessage)
assert connectionRequestResponse.text != "", "invalid response from acapy"

**Response from aca-py**

In [45]:
# a bit of a hack to simplify message unpacking,
# decode_pack_message needs a callable object for some reason
def unpackMessage(message, privateKey):
    class FindKey:
        def __init__(self, key):
            self.key = key

        def __call__(self, argument):
            return self.key

    find_key = FindKey(privateKey)   
    return crypto.decode_pack_message(message, find_key)
    
connectionRequestResponseUnpacked = \
    unpackMessage(connectionRequestResponse.text, plugin['keypair'][1])

connectionRequestResponseDict = json.loads(connectionRequestResponseUnpacked[0])

print("Decoded acapy response: \n")
pp.pprint(connectionRequestResponseUnpacked)

Decoded acapy response: 

('{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/response", '
 '"@id": "2bdec4f9-1292-4eff-bffe-24e46ba7bec1", "~thread": {"thid": '
 '"db6ec1f7-46fc-4e12-bc2e-7ab797e3222e"}, "connection~sig": {"@type": '
 '"did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/signature/1.0/ed25519Sha512_single", '
 '"signature": '
 '"5RTrrJnWP0kG7b3peYjcnqAnvE6A_7XBW2Iun9H8P_pF63WyWgkoRonlKTYXvAdRoNbWoYJhynAHGQxXLsYfDg==", '
 '"sig_data": '
 '"AAAAAF67pih7IkRJRCI6ICJLRFVNb3BwZkYzakhaMWQxNnloaHh4IiwgIkRJRERvYyI6IHsiQGNvbnRleHQiOiAiaHR0cHM6Ly93M2lkLm9yZy9kaWQvdjEiLCAiaWQiOiAiZGlkOnNvdjpLRFVNb3BwZkYzakhaMWQxNnloaHh4IiwgInB1YmxpY0tleSI6IFt7ImlkIjogImRpZDpzb3Y6S0RVTW9wcGZGM2pIWjFkMTZ5aGh4eCMxIiwgInR5cGUiOiAiRWQyNTUxOVZlcmlmaWNhdGlvbktleTIwMTgiLCAiY29udHJvbGxlciI6ICJkaWQ6c292OktEVU1vcHBmRjNqSFoxZDE2eWhoeHgiLCAicHVibGljS2V5QmFzZTU4IjogIkF2cDl0VUZBZGFYTjJ2ZU1SdHJHVHBORXpyMzdhTHN2cHhaSGZXWGNUQkF2In1dLCAiYXV0aGVudGljYXRpb24iOiBbeyJ0eXBlIjogIkVkMjU1MTlTaWduYXR1cmVBdXRoZW50aWNhdGlvbjIwMT

**Unpacking connection data embedded in the response**

In [46]:
sig_data_raw = connectionRequestResponseDict['connection~sig']['sig_data']

# sig_data_raw = sig_data_raw[10:]
# print(sig_data_raw)
# aca-py responses seem to be encoded (so far) in base64
sig_data_raw = base64.b64decode(sig_data_raw)
# print(sig_data_raw)
# avoid first 8 characters as they are a time signature
sig_data_raw = sig_data_raw[8:]
sig_data_raw = sig_data_raw.decode('ascii')

sig_data = json.loads(sig_data_raw)
pp.pprint(sig_data)

{'DID': 'KDUMoppfF3jHZ1d16yhhxx',
 'DIDDoc': {'@context': 'https://w3id.org/did/v1',
            'authentication': [{'publicKey': 'did:sov:KDUMoppfF3jHZ1d16yhhxx#1',
                                'type': 'Ed25519SignatureAuthentication2018'}],
            'id': 'did:sov:KDUMoppfF3jHZ1d16yhhxx',
            'publicKey': [{'controller': 'did:sov:KDUMoppfF3jHZ1d16yhhxx',
                           'id': 'did:sov:KDUMoppfF3jHZ1d16yhhxx#1',
                           'publicKeyBase58': 'Avp9tUFAdaXN2veMRtrGTpNEzr37aLsvpxZHfWXcTBAv',
                           'type': 'Ed25519VerificationKey2018'}],
            'service': [{'id': 'did:sov:KDUMoppfF3jHZ1d16yhhxx;indy',
                         'priority': 0,
                         'recipientKeys': ['Avp9tUFAdaXN2veMRtrGTpNEzr37aLsvpxZHfWXcTBAv'],
                         'serviceEndpoint': 'http://agent2.localhost',
                         'type': 'IndyAgent'}]}}


**Adding the connection to connection list**

In [47]:
connections = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": sig_data['DIDDoc'],
        "myKey": plugin['keypair'][1]
    }
]

def packMessage(message, connection):
    # pass in our private key and recipient key to the encode_pack_message
    decodedRecipientKey = base58.b58decode(connection['DIDDoc']['service'][0]['recipientKeys'][0])
    packedMessage = \
    crypto.encode_pack_message(json.dumps(message), \
                               [decodedRecipientKey], \
                               connection['myKey'])
    return packedMessage.decode('ascii')


pp.pprint(connections)

[{'DIDDoc': {'@context': 'https://w3id.org/did/v1',
             'authentication': [{'publicKey': 'did:sov:KDUMoppfF3jHZ1d16yhhxx#1',
                                 'type': 'Ed25519SignatureAuthentication2018'}],
             'id': 'did:sov:KDUMoppfF3jHZ1d16yhhxx',
             'publicKey': [{'controller': 'did:sov:KDUMoppfF3jHZ1d16yhhxx',
                            'id': 'did:sov:KDUMoppfF3jHZ1d16yhhxx#1',
                            'publicKeyBase58': 'Avp9tUFAdaXN2veMRtrGTpNEzr37aLsvpxZHfWXcTBAv',
                            'type': 'Ed25519VerificationKey2018'}],
             'service': [{'id': 'did:sov:KDUMoppfF3jHZ1d16yhhxx;indy',
                          'priority': 0,
                          'recipientKeys': ['Avp9tUFAdaXN2veMRtrGTpNEzr37aLsvpxZHfWXcTBAv'],
                          'serviceEndpoint': 'http://agent2.localhost',
                          'type': 'IndyAgent'}]},
  'label': 'Client (admin)',
  'myKey': b'_\xab\xc5\xd6D\x9a\xa1\x08\x89\xa5\x8b\xb1+\x15\x99\xd

### FEATURE DISCOVERY

In [48]:
message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/query"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
message['query'] = '*'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'fdb775f4-8e87-4a88-be7e-c9bad8e7f5f3',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/disclose',
 'protocols': [{'pid': 'https://didcomm.org/trust_ping/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/routing/1.0'},
               {'pid': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-connections/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/action-menu/1.0',
                'roles': ['provider']},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/introduction-service/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-holder/0.1'},
               {'pid': 'https://didcomm.org/action-menu/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/notification/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-static-connections/0.1'},
               {'p

## Dids

In [49]:
message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-list-dids"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded['result'])

[{'did': 'FfA9ucEJWhy7VSk56NrNsj',
  'verkey': '8zNAZ12Xqmzdht3E4UtRSy94Eo4TUFHVxkzNpAezHnhM'},
 {'did': 'FKZ2qzNnESHLkULHaNMVr',
  'verkey': '8ogPtgR5PHsBSSG2KNc3tjmqjCBDk1Yg5btefWwS9Vu'},
 {'did': 'KDUMoppfF3jHZ1d16yhhxx',
  'verkey': 'Avp9tUFAdaXN2veMRtrGTpNEzr37aLsvpxZHfWXcTBAv'},
 {'did': 'YU8iw8LBTQasfJp2dFJnoq',
  'verkey': 'J9m4JzAWukBnTotGSVgm1EDcj1ni5HcKm3rbsHVKvcMi'}]


## Public did

In [50]:
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-public-did"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '27ea603f-08dd-4ca2-9030-b22664712b14',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 '~thread': {'thid': '432e6ac7-516f-4259-8b86-e943d27e0404'}}


In [51]:

message = {}
message['@id'] = str(uuid.uuid4())
message['@type'] = "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/hello-world/1.0/query"
message['~transport'] = {}
message['~transport']['return_route'] = 'all'
message['query'] = '*'
# message['comment'] = "memes"
encodedMessage = packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': 'f2a5a4d8-ace2-48fb-b37d-626874a023e4',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/hello-world/1.0/disclose',
 'protocols': 'hello world',
 '~thread': {'thid': '96e5f956-22c1-4e15-b2e9-efb4271fda7a'}}
